# Data Augmentation Strategy - Expansión Inteligente del Dataset
**Sistema de Anotación de Video - Entrega 2**

## Análisis del EDA Actualizado (90 videos):
- **Dataset base:** 6,443 frames (90 videos)
- **Balance ratio actual:** 0.51 (Girar: 942 / Caminar Hacia: 1,844)
- **Desbalance:** MODERADO (objetivo: >0.90)
- **Distribución:**
  - Caminar Hacia: 1,844 frames (28.6%)
  - Caminar Regreso: 1,301 frames (20.2%)
  - Sentarse: 1,253 frames (19.4%)
  - Ponerse Pie: 1,103 frames (17.1%)
  - Girar: 942 frames (14.6%)

## Objetivo:
- Balancear a ratio >0.90
- Target: ~1,660 frames por actividad
- Frames a generar: ~902 adicionales
- Dataset final: ~7,345 frames

## Estrategia:
1. SMOTE para balanceo (prioridad a Girar)
2. Rotaciones espaciales (robustez)
3. Interpolación temporal (opcional)


## Paso 1: Instalación y Setup
Instalar librerías necesarias para data augmentation.


In [1]:
# Instalar dependencias para augmentation
!pip install pandas numpy matplotlib seaborn scikit-learn imbalanced-learn scipy

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn # Import imblearn directly
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline
from pathlib import Path
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías para augmentation cargadas")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"🔬 Imbalanced-learn: {imblearn.__version__}")

✅ Librerías para augmentation cargadas
📊 Pandas: 2.2.2
🔢 NumPy: 2.0.2
🔬 Imbalanced-learn: 0.14.0


## Paso 2: Cargar Dataset Base
Cargar el dataset limpio de la Entrega 1 para análisis de desbalance.


In [2]:
# CARGAR DATASET BASE DE ENTREGA 1
def load_base_dataset():
    """Cargar dataset base desde Entrega 1"""
    print("📂 CARGANDO DATASET BASE DESDE ENTREGA 1")
    print("=" * 50)

    # Ruta relativa desde Entrega2 hacia Entrega1 - Adjusting to load from current directory
    landmarks_path = Path(".") # Search in the current directory

    if not landmarks_path.exists():
        print(f"❌ No se encontró: {landmarks_path}")
        print("💡 Asegúrate de ejecutar desde Entrega2/notebooks/")
        return None

    # Cargar todos los CSVs ending with _landmarks.csv
    csv_files = list(landmarks_path.glob("*_landmarks.csv"))
    print(f"📁 Archivos encontrados: {len(csv_files)}")

    if not csv_files:
        print("❌ No se encontraron archivos CSV de landmarks en el directorio actual.")
        return None

    dataframes = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)

    # Limpiar datos (remover frames sin detección)
    landmark_cols = [col for col in combined_df.columns
                    if col not in ['activity', 'video_file', 'frame_number']]

    # Solo frames con detección
    df_clean = combined_df[(combined_df[landmark_cols] != 0.0).any(axis=1)].copy()

    print(f"📊 DATASET CARGADO:")
    print(f"   Total frames: {len(df_clean):,}")
    print(f"   Videos: {df_clean['video_file'].nunique()}")
    print(f"   Actividades: {df_clean['activity'].nunique()}")

    # Mostrar distribución actual
    print(f"\n📈 DISTRIBUCIÓN ACTUAL:")
    activity_counts = df_clean['activity'].value_counts()
    for activity, count in activity_counts.items():
        pct = count / len(df_clean) * 100
        print(f"   {activity.replace('_', ' ').title()}: {count:,} frames ({pct:.1f}%)")

    balance_ratio = activity_counts.min() / activity_counts.max()
    print(f"\n⚖️ Balance actual: {balance_ratio:.2f}")

    return df_clean

# Cargar dataset base
base_df = load_base_dataset()

📂 CARGANDO DATASET BASE DESDE ENTREGA 1
📁 Archivos encontrados: 90
📊 DATASET CARGADO:
   Total frames: 6,443
   Videos: 90
   Actividades: 5

📈 DISTRIBUCIÓN ACTUAL:
   Caminar Hacia: 1,844 frames (28.6%)
   Caminar Regreso: 1,301 frames (20.2%)
   Sentarse: 1,253 frames (19.4%)
   Ponerse Pie: 1,103 frames (17.1%)
   Girar: 942 frames (14.6%)

⚖️ Balance actual: 0.51


In [4]:
# ============================================
# PASO 2.5: SPLIT ESTRATIFICADO (INSERTAR ANTES DE PASO 4)
# ============================================

from sklearn.model_selection import train_test_split
import numpy as np

def split_dataset_before_augmentation(df):
    """
    Split estratificado ANTES de augmentation para evitar leakage
    """
    print("\n🔀 SPLIT ESTRATIFICADO DEL DATASET")
    print("=" * 60)

    # Preparar features y labels
    landmark_cols = [col for col in df.columns
                    if col not in ['activity', 'video_file', 'frame_number']]

    X = df[landmark_cols]
    y = df['activity']

    # Split: 70% train, 15% validation, 15% test
    # Primero: separar test (15%)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y,
        test_size=0.15,
        stratify=y,
        random_state=42
    )

    # Segundo: separar train y validation del 85% restante
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=0.1765,  # 15% del total = 17.65% del 85%
        stratify=y_temp,
        random_state=42
    )

    print(f"📊 DISTRIBUCIÓN DEL SPLIT:")
    print(f"   Train:      {len(X_train):,} frames ({len(X_train)/len(df)*100:.1f}%)")
    print(f"   Validation: {len(X_val):,} frames ({len(X_val)/len(df)*100:.1f}%)")
    print(f"   Test:       {len(X_test):,} frames ({len(X_test)/len(df)*100:.1f}%)")

    # Verificar balance en cada split
    print(f"\n⚖️ BALANCE POR SPLIT:")

    for split_name, y_split in [('Train', y_train), ('Validation', y_val), ('Test', y_test)]:
        counts = y_split.value_counts()
        ratio = counts.min() / counts.max()
        print(f"\n   {split_name}:")
        for activity, count in counts.items():
            pct = count / len(y_split) * 100
            print(f"      {activity:20s}: {count:4d} ({pct:4.1f}%)")
        print(f"      Balance ratio: {ratio:.3f}")

    # Crear DataFrames completos
    train_df = pd.concat([X_train, y_train], axis=1)
    val_df = pd.concat([X_val, y_val], axis=1)
    test_df = pd.concat([X_test, y_test], axis=1)

    # Agregar metadata
    train_df['split'] = 'train'
    val_df['split'] = 'validation'
    test_df['split'] = 'test'

    print(f"\n✅ SPLIT COMPLETADO")
    print(f"💡 IMPORTANTE: Augmentation se aplicará SOLO al train set")

    return train_df, val_df, test_df

# Ejecutar split
if base_df is not None:
    train_df, val_df, test_df = split_dataset_before_augmentation(base_df)

    print(f"\n📁 SETS CREADOS:")
    print(f"   train_df: {len(train_df):,} frames (para augmentation)")
    print(f"   val_df:   {len(val_df):,} frames (sin tocar)")
    print(f"   test_df:  {len(test_df):,} frames (sin tocar)")
else:
    print("❌ No hay dataset base")



🔀 SPLIT ESTRATIFICADO DEL DATASET
📊 DISTRIBUCIÓN DEL SPLIT:
   Train:      4,509 frames (70.0%)
   Validation: 967 frames (15.0%)
   Test:       967 frames (15.0%)

⚖️ BALANCE POR SPLIT:

   Train:
      caminar_hacia       : 1290 (28.6%)
      caminar_regreso     :  911 (20.2%)
      sentarse            :  877 (19.4%)
      ponerse_pie         :  771 (17.1%)
      girar               :  660 (14.6%)
      Balance ratio: 0.512

   Validation:
      caminar_hacia       :  277 (28.6%)
      caminar_regreso     :  195 (20.2%)
      sentarse            :  188 (19.4%)
      ponerse_pie         :  166 (17.2%)
      girar               :  141 (14.6%)
      Balance ratio: 0.509

   Test:
      caminar_hacia       :  277 (28.6%)
      caminar_regreso     :  195 (20.2%)
      sentarse            :  188 (19.4%)
      ponerse_pie         :  166 (17.2%)
      girar               :  141 (14.6%)
      Balance ratio: 0.509

✅ SPLIT COMPLETADO
💡 IMPORTANTE: Augmentation se aplicará SOLO al train set

📁

## Paso 3: Análisis de Desbalance
Analizar en detalle el desbalance de clases y calcular targets de augmentation.


In [6]:
# ============================================
# PASO 3 MODIFICADO: ANÁLISIS SOLO DEL TRAIN SET
# ============================================

def analyze_train_imbalance(train_df):
    """Analizar desbalance SOLO del train set"""
    print("\n⚖️ ANÁLISIS DE DESBALANCE - TRAIN SET")
    print("=" * 60)

    activity_counts = train_df['activity'].value_counts()

    # Calcular target conservador (80% de la clase mayoritaria)
    # Esto evita over-augmentation
    target_frames = int(activity_counts.max() * 0.80)

    print(f"🎯 ESTRATEGIA CONSERVADORA:")
    print(f"   Clase mayoritaria: {activity_counts.max():,} frames")
    print(f"   Target ajustado: {target_frames:,} frames (80% de max)")
    print(f"   Razón: Evitar overfitting a datos sintéticos")

    augmentation_needed = {}
    total_augmentation = 0

    print(f"\n📊 PLAN DE BALANCEO PARA TRAIN SET:")

    for activity, current_count in activity_counts.items():
        needed = max(0, target_frames - current_count)
        augmentation_needed[activity] = needed
        total_augmentation += needed

        if needed > 0:
            print(f"   {activity:20s}: {current_count:4d} → {target_frames:4d} (+{needed:3d})")
        else:
            print(f"   {activity:20s}: {current_count:4d} (✅ OK)")

    print(f"\n🎯 RESUMEN:")
    print(f"   Train original: {len(train_df):,} frames")
    print(f"   Frames a generar: {total_augmentation:,}")
    print(f"   Train final: {len(train_df) + total_augmentation:,} frames")
    print(f"   Ratio sintético: {(total_augmentation/(len(train_df)+total_augmentation))*100:.1f}%")

    # Verificar que ratio sintético < 20%
    if (total_augmentation / (len(train_df) + total_augmentation)) > 0.20:
        print(f"   ⚠️  WARNING: Ratio sintético alto, considerar reducir target")
    else:
        print(f"   ✅ Ratio sintético seguro (<20%)")

    return augmentation_needed, target_frames

# Ejecutar análisis del train set
if 'train_df' in locals():
    train_aug_needed, train_target = analyze_train_imbalance(train_df)
else:
    print("❌ Primero ejecutar split del dataset")



⚖️ ANÁLISIS DE DESBALANCE - TRAIN SET
🎯 ESTRATEGIA CONSERVADORA:
   Clase mayoritaria: 1,290 frames
   Target ajustado: 1,032 frames (80% de max)
   Razón: Evitar overfitting a datos sintéticos

📊 PLAN DE BALANCEO PARA TRAIN SET:
   caminar_hacia       : 1290 (✅ OK)
   caminar_regreso     :  911 → 1032 (+121)
   sentarse            :  877 → 1032 (+155)
   ponerse_pie         :  771 → 1032 (+261)
   girar               :  660 → 1032 (+372)

🎯 RESUMEN:
   Train original: 4,509 frames
   Frames a generar: 909
   Train final: 5,418 frames
   Ratio sintético: 16.8%
   ✅ Ratio sintético seguro (<20%)


## Paso 4: Implementar SMOTE para Balanceo
Aplicar SMOTE (Synthetic Minority Oversampling Technique) para generar datos sintéticos de clases minoritarias.


In [7]:
# ============================================
# PASO 4: APLICAR SMOTE AL TRAIN SET
# ============================================

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

def apply_smote_to_train(train_df, target_per_class=1032):
    """
    Aplicar SMOTE SOLO al train set según el plan de balanceo
    """
    print("\n🔄 APLICANDO SMOTE AL TRAIN SET")
    print("=" * 60)

    # Separar features y labels
    landmark_cols = [col for col in train_df.columns
                    if col not in ['activity', 'video_file', 'frame_number', 'split']]

    X_train = train_df[landmark_cols].values
    y_train = train_df['activity'].values

    # Mostrar distribución ANTES
    print("📊 DISTRIBUCIÓN ANTES DE SMOTE:")
    unique, counts = np.unique(y_train, return_counts=True)
    for activity, count in zip(unique, counts):
        print(f"   {activity:20s}: {count:4d} frames")

    original_balance = counts.min() / counts.max()
    print(f"   Balance ratio: {original_balance:.3f}")

    # Calcular estrategia de sampling
    activity_counts = train_df['activity'].value_counts()
    sampling_strategy = {}

    for activity in activity_counts.index:
        current_count = activity_counts[activity]
        if current_count < target_per_class:
            sampling_strategy[activity] = target_per_class

    print(f"\n🎯 ESTRATEGIA SMOTE:")
    for activity, target in sampling_strategy.items():
        current = activity_counts[activity]
        print(f"   {activity:20s}: {current:4d} → {target:4d} (+{target-current:3d})")

    # Aplicar SMOTE
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_train)

    # Convertir sampling_strategy a índices
    sampling_strategy_encoded = {
        label_encoder.transform([activity])[0]: target
        for activity, target in sampling_strategy.items()
    }

    smote = SMOTE(
        sampling_strategy=sampling_strategy_encoded,
        random_state=42,
        k_neighbors=min(5, min(counts)-1)  # Adaptativo
    )

    try:
        print(f"\n⏳ Generando datos sintéticos con SMOTE...")
        X_train_balanced, y_encoded_balanced = smote.fit_resample(X_train, y_encoded)
        y_train_balanced = label_encoder.inverse_transform(y_encoded_balanced)

        # Mostrar distribución DESPUÉS
        print(f"\n📊 DISTRIBUCIÓN DESPUÉS DE SMOTE:")
        unique_after, counts_after = np.unique(y_train_balanced, return_counts=True)

        total_generated = 0
        for activity, count in zip(unique_after, counts_after):
            original_count = activity_counts[activity]
            generated = count - original_count
            total_generated += generated

            if generated > 0:
                print(f"   {activity:20s}: {original_count:4d} → {count:4d} (+{generated:3d} sintéticos)")
            else:
                print(f"   {activity:20s}: {count:4d} (sin cambios)")

        new_balance = counts_after.min() / counts_after.max()

        print(f"\n📈 RESUMEN:")
        print(f"   Frames originales: {len(X_train):,}")
        print(f"   Frames generados: {total_generated:,}")
        print(f"   Train final: {len(X_train_balanced):,}")
        print(f"   Ratio sintético: {(total_generated/len(X_train_balanced))*100:.1f}%")
        print(f"   Balance anterior: {original_balance:.3f}")
        print(f"   Balance nuevo: {new_balance:.3f}")
        print(f"   Mejora: {((new_balance-original_balance)/original_balance)*100:+.1f}%")

        # Crear DataFrame balanceado
        train_balanced_df = pd.DataFrame(X_train_balanced, columns=landmark_cols)
        train_balanced_df['activity'] = y_train_balanced
        train_balanced_df['split'] = 'train'

        # Marcar datos sintéticos
        train_balanced_df['data_type'] = ['original'] * len(X_train) + \
                                         ['smote'] * (len(X_train_balanced) - len(X_train))

        print(f"\n✅ SMOTE APLICADO EXITOSAMENTE AL TRAIN SET")

        return train_balanced_df, total_generated

    except Exception as e:
        print(f"\n❌ Error aplicando SMOTE: {e}")
        return None, 0

# Ejecutar SMOTE
if 'train_df' in locals() and train_df is not None:
    train_balanced, smote_generated = apply_smote_to_train(train_df, target_per_class=1032)

    if train_balanced is not None:
        print(f"\n🎉 Train set balanceado listo para entrenamiento")
else:
    print("❌ Primero debes tener train_df del split")



🔄 APLICANDO SMOTE AL TRAIN SET
📊 DISTRIBUCIÓN ANTES DE SMOTE:
   caminar_hacia       : 1290 frames
   caminar_regreso     :  911 frames
   girar               :  660 frames
   ponerse_pie         :  771 frames
   sentarse            :  877 frames
   Balance ratio: 0.512

🎯 ESTRATEGIA SMOTE:
   caminar_regreso     :  911 → 1032 (+121)
   sentarse            :  877 → 1032 (+155)
   ponerse_pie         :  771 → 1032 (+261)
   girar               :  660 → 1032 (+372)

⏳ Generando datos sintéticos con SMOTE...

📊 DISTRIBUCIÓN DESPUÉS DE SMOTE:
   caminar_hacia       : 1290 (sin cambios)
   caminar_regreso     :  911 → 1032 (+121 sintéticos)
   girar               :  660 → 1032 (+372 sintéticos)
   ponerse_pie         :  771 → 1032 (+261 sintéticos)
   sentarse            :  877 → 1032 (+155 sintéticos)

📈 RESUMEN:
   Frames originales: 4,509
   Frames generados: 909
   Train final: 5,418
   Ratio sintético: 16.8%
   Balance anterior: 0.512
   Balance nuevo: 0.800
   Mejora: +56.4%

✅ SMOTE

## Paso 7: Combinar Dataset Final Aumentado
Combinar todos los datos aumentados (original + SMOTE + rotaciones + interpolación) en dataset final.


In [11]:
# ============================================
# PASO 7: GUARDAR DATASETS FINALES (CORREGIDO)
# ============================================

import os
import json
from datetime import datetime

def save_final_datasets():
    """
    Guardar train/val/test en archivos separados
    """
    print("\n💾 GUARDANDO DATASETS FINALES")
    print("=" * 60)

    # Crear directorio
    output_dir = 'data/processed'
    os.makedirs(output_dir, exist_ok=True)

    # Rutas de archivos
    train_path = f'{output_dir}/train_balanced.csv'
    val_path = f'{output_dir}/val_original.csv'
    test_path = f'{output_dir}/test_original.csv'

    # Guardar CSVs
    print(f"\n📁 Guardando archivos...")
    train_balanced.to_csv(train_path, index=False)
    val_df.to_csv(val_path, index=False)
    test_df.to_csv(test_path, index=False)

    # Calcular estadísticas
    train_balance = train_balanced['activity'].value_counts().min() / train_balanced['activity'].value_counts().max()
    val_balance = val_df['activity'].value_counts().min() / val_df['activity'].value_counts().max()
    test_balance = test_df['activity'].value_counts().min() / test_df['activity'].value_counts().max()

    # Balance ORIGINAL (antes de SMOTE) - AÑADIDO
    original_balance = 0.512  # Del análisis inicial

    # Contar sintéticos
    smote_count = len(train_balanced[train_balanced['data_type'] == 'smote'])
    original_train_count = len(train_balanced[train_balanced['data_type'] == 'original'])

    print(f"✅ ARCHIVOS GUARDADOS:")
    print(f"\n   📁 {train_path}")
    print(f"      Total: {len(train_balanced):,} frames")
    print(f"      • Original: {original_train_count:,} (83.2%)")
    print(f"      • SMOTE: {smote_count:,} (16.8%)")
    print(f"      • Balance: {train_balance:.3f}")

    print(f"\n   📁 {val_path}")
    print(f"      Total: {len(val_df):,} frames")
    print(f"      • 100% datos reales")
    print(f"      • Balance: {val_balance:.3f}")

    print(f"\n   📁 {test_path}")
    print(f"      Total: {len(test_df):,} frames")
    print(f"      • 100% datos reales")
    print(f"      • Balance: {test_balance:.3f}")

    # Tamaños de archivos
    train_size = os.path.getsize(train_path) / (1024**2)
    val_size = os.path.getsize(val_path) / (1024**2)
    test_size = os.path.getsize(test_path) / (1024**2)
    total_size = train_size + val_size + test_size

    print(f"\n💾 TAMAÑOS DE ARCHIVOS:")
    print(f"   Train: {train_size:.2f} MB")
    print(f"   Validation: {val_size:.2f} MB")
    print(f"   Test: {test_size:.2f} MB")
    print(f"   Total: {total_size:.2f} MB")

    # Crear metadata
    metadata = {
        'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'project': 'Sistema de Anotación de Video - Entrega 2',
        'total_frames': len(train_balanced) + len(val_df) + len(test_df),
        'augmentation_strategy': 'SMOTE only (no leakage)',
        'splits': {
            'train': {
                'file': 'train_balanced.csv',
                'frames': len(train_balanced),
                'original_frames': original_train_count,
                'smote_frames': smote_count,
                'balance_ratio': float(train_balance),
                'percentage': 70.0,
                'activities': train_balanced['activity'].value_counts().to_dict()
            },
            'validation': {
                'file': 'val_original.csv',
                'frames': len(val_df),
                'data_type': '100% real',
                'balance_ratio': float(val_balance),
                'percentage': 15.0,
                'activities': val_df['activity'].value_counts().to_dict()
            },
            'test': {
                'file': 'test_original.csv',
                'frames': len(test_df),
                'data_type': '100% real',
                'balance_ratio': float(test_balance),
                'percentage': 15.0,
                'activities': test_df['activity'].value_counts().to_dict()
            }
        },
        'quality_checks': {
            'no_data_leakage': True,
            'synthetic_ratio_train': float(smote_count / len(train_balanced)),
            'synthetic_ratio_total': float(smote_count / (len(train_balanced) + len(val_df) + len(test_df))),
            'balance_improvement': f"{original_balance:.3f} → {train_balance:.3f} (+{((train_balance-original_balance)/original_balance)*100:.1f}%)"
        }
    }

    # Guardar metadata
    metadata_path = f'{output_dir}/metadata.json'
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)

    print(f"\n📋 Metadata guardada: {metadata_path}")

    # Crear README
    readme_path = f'{output_dir}/README.txt'
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write("=" * 60 + "\n")
        f.write("DATASET DE LANDMARKS - PROYECTO IA1 ENTREGA 2\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("ARCHIVOS:\n")
        f.write(f"• train_balanced.csv: {len(train_balanced):,} frames (con SMOTE)\n")
        f.write(f"• val_original.csv: {len(val_df):,} frames (100% real)\n")
        f.write(f"• test_original.csv: {len(test_df):,} frames (100% real)\n\n")
        f.write("CARACTERÍSTICAS:\n")
        f.write(f"• Balance train: {train_balance:.3f}\n")
        f.write(f"• Ratio sintético train: {(smote_count/len(train_balanced))*100:.1f}%\n")
        f.write(f"• Val/Test: 100% datos reales\n")
        f.write("• Sin data leakage: SMOTE aplicado solo a train\n\n")
        f.write("USO EN NOTEBOOK 4 (MODELADO):\n")
        f.write("1. train_balanced.csv → Entrenamiento + cross-validation\n")
        f.write("2. val_original.csv → Ajuste de hiperparámetros\n")
        f.write("3. test_original.csv → Evaluación final ÚNICA\n")

    print(f"📄 README creado: {readme_path}")

    print(f"\n✅ GUARDADO COMPLETADO")

    return {
        'train': train_path,
        'val': val_path,
        'test': test_path,
        'metadata': metadata_path
    }

# EJECUTAR GUARDADO
paths = save_final_datasets()

if paths:
    print(f"\n" + "🎉" * 30)
    print(f"🎉 NOTEBOOK 3 - DATA AUGMENTATION COMPLETADO 🎉")
    print("🎉" * 30)

    print(f"\n📊 RESUMEN EJECUTIVO:")
    print(f"   Dataset original: 6,443 frames")
    print(f"   Dataset final: 7,352 frames")
    print(f"   • Train: 5,418 (con SMOTE)")
    print(f"   • Validation: 967 (real)")
    print(f"   • Test: 967 (real)")

    print(f"\n✅ GARANTÍAS DE CALIDAD:")
    print(f"   ✓ Sin data leakage")
    print(f"   ✓ Balance mejorado: 0.512 → 0.800")
    print(f"   ✓ Ratio sintético seguro: 16.8%")
    print(f"   ✓ Val/Test 100% reales")

    print(f"\n📁 ARCHIVOS LISTOS EN: data/processed/")
    print(f"   • train_balanced.csv")
    print(f"   • val_original.csv")
    print(f"   • test_original.csv")
    print(f"   • metadata.json")
    print(f"   • README.txt")

    print(f"\n🚀 SIGUIENTE PASO:")
    print(f"   Notebook 4: Entrenar modelos con train_balanced.csv")



💾 GUARDANDO DATASETS FINALES

📁 Guardando archivos...
✅ ARCHIVOS GUARDADOS:

   📁 data/processed/train_balanced.csv
      Total: 5,418 frames
      • Original: 4,509 (83.2%)
      • SMOTE: 909 (16.8%)
      • Balance: 0.800

   📁 data/processed/val_original.csv
      Total: 967 frames
      • 100% datos reales
      • Balance: 0.509

   📁 data/processed/test_original.csv
      Total: 967 frames
      • 100% datos reales
      • Balance: 0.509

💾 TAMAÑOS DE ARCHIVOS:
   Train: 6.44 MB
   Validation: 1.14 MB
   Test: 1.14 MB
   Total: 8.72 MB

📋 Metadata guardada: data/processed/metadata.json
📄 README creado: data/processed/README.txt

✅ GUARDADO COMPLETADO

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
🎉 NOTEBOOK 3 - DATA AUGMENTATION COMPLETADO 🎉
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

📊 RESUMEN EJECUTIVO:
   Dataset original: 6,443 frames
   Dataset final: 7,352 frames
   • Train: 5,418 (con SMOTE)
   • Validation: 967 (real)
   • Test: 967 (real)

✅ GARANTÍAS DE CALIDAD:
   ✓ Sin data leakage
   ✓ Balance